In [1]:
import typing
import openpyxl
import datetime
import sqlalchemy
import numpy as np

from pandas import read_excel
from pandas import DataFrame

In [24]:
path = r'/home/bradleyk/Downloads/Dashboard - Sales - Excel.xlsx'

x = openpyxl.load_workbook(path, read_only=True)
# y.iter_cols

In [25]:
#sheet: typing.Union[str, int]
class Transformation:
    def __init__(self, func) -> None:        
        self.func = func

    def apply_transform(
        self, 
        value: typing.Union[int, float, bool, str], 
        raise_exception: bool=False) -> typing.Union[int, float, bool, str]:
        
        try:
            value = self.func(value)
        except Exception as e:
            if raise_exception:
                raise e
                
            return None
        
        return value
        

class Field:
    TYPE_MAPPING = {
        'n': float,
        's': str,
        'd': datetime.date}
    
    def __init__(
            self,
            name: str,
            dtype: typing.Union[int, float, bool, str],
            use_internal_value: bool=False) -> None:
        """
        Args:
            name: The name to be applied to the field
            data_type: A data type to be applied to the field"""
        
        self.name = name
        self._dtype = dtype
        self.use_internal_value = use_internal_value
        
        # List of transformations
        self.pipeline: typing.List[Transformation] = []
    
    @property
    def dtype(self) -> typing.Union[int, float, bool, str]:
        return self._dtype

    @dtype.setter
    def dtype(self, value: typing.Union[int, float, bool, str]):
        if value not in (int, float, bool, str):
            raise ValueError(f"Invalid dtype")
            
        self.pipeline.append(Transformation(value))
        self._dtype = value
    
    def transform(self, cell, raise_exception: bool=False):
        if self.use_internal_value:
            value = cell.internal_value
        else:
            value = cell.value
            
        for pipe in self.pipeline:
            value = pipe.apply_transform(value, raise_exception)
        
        return value


class Table:
    def __init__(self, worksheet):
        self.error = None
        self._worksheet = worksheet
        
        rows = self._worksheet.iter_rows()
        headers = next(rows)
        sample = next(rows)
        
        self.columns = []
        for header, row in zip(headers, sample):
            try:
                name = header.value
                dtype = Field.TYPE_MAPPING[row.data_type]
                
                # Create a column object
                column = Field(name, dtype)
                self.columns.append(column)
                
                setattr(self, name, column)
            except Exception as e:
                self.error = f"Error encountered in creating table object - {e}"
                
                for column in self.columns:
                    if column.name is not None:
                        delattr(self, column.name)
                
                break

    def __getitem__(self, name):
        return getattr(self, name)

    @property
    def rows(self):
        for row in self._worksheet.iter_rows(row_offset=1):
            yield row
            
    def read(self, raise_exception: bool=False) -> typing.List:        
        data = []
        for row in self.rows:
            data.append(
                [column.transform(cell, raise_exception) for column, cell in zip(self.columns, row)])
            
        return data
    
    def to_frame(self, raise_exception: bool=False) -> DataFrame:
        columns = [column.name for column in self.columns]
        return DataFrame(self.read(raise_exception), columns=columns)

    
class Excel:
    def __init__(self, path: str, raise_exception: bool=False):
        self._workbook = openpyxl.load_workbook(path, read_only=True)
        self.raise_exception = raise_exception
        
        for name in self._workbook.get_sheet_names():
            worksheet = self._workbook.get_sheet_by_name(name)
            
            setattr(self, name, Table(worksheet))
    
    def close(self):
        self._workbook.close()
        
    def __getitem__(self, name):
        return getattr(self, name)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._workbook.close()


In [27]:
with Excel(path) as excel:
    data = excel.Orders.to_frame()

data.head()

,Order ID,Order Date,Customer,Customer Country,Supplier,Shipper,Amount,Discount,Quantity
0,10370,1/3/1995,Chop-suey Chinese,Switzerland,Exotic Liquids,United Package,183.599999,32.400000,15
1,10370,1/3/1995,Chop-suey Chinese,Switzerland,Plutzer Lebensmittelgroßmärkte AG,United Package,798.000000,0.000000,30
2,10370,1/3/1995,Chop-suey Chinese,Switzerland,Tokyo Traders,United Package,135.999999,24.000000,20
3,10371,1/3/1995,La maison d'Asie,France,Svensk Sjöföda AB,Speedy Express,72.960000,18.240001,6
4,10373,1/5/1995,Hungry Owl All-Night Grocers,Ireland,Escargots Nouveaux,Federal Shipping,678.399997,169.600000,80


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 9 columns):
Order ID            247 non-null int64
Order Date          247 non-null object
Customer            247 non-null object
Customer Country    247 non-null object
Supplier            247 non-null object
Shipper             247 non-null object
Amount              247 non-null float64
Discount            247 non-null float64
Quantity            247 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 17.4+ KB


In [28]:
%%time
with Excel(path) as excel:
    
    # Order ID ETL
    excel.Orders['Order ID'].name = 'order_id'
    excel.Orders['Order ID'].dtype = int
    
    # Order date ETL
    excel.Orders['Order Date'].name = 'long_date'

    parse_date = Transformation(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))
    excel.Orders['Order Date'].pipeline.append(parse_date)
        
    render_date = Transformation(lambda x: datetime.datetime.strftime(x, '%B %Y'))
    excel.Orders['Order Date'].pipeline.append(render_date)
    
    data = excel.Orders.to_frame(True)

data.head()

CPU times: user 60.7 ms, sys: 0 ns, total: 60.7 ms
Wall time: 60.3 ms


In [30]:
data.head()

,order_id,long_date,Customer,Customer Country,Supplier,Shipper,Amount,Discount,Quantity
0,10370,January 1995,Chop-suey Chinese,Switzerland,Exotic Liquids,United Package,183.599999,32.400000,15
1,10370,January 1995,Chop-suey Chinese,Switzerland,Plutzer Lebensmittelgroßmärkte AG,United Package,798.000000,0.000000,30
2,10370,January 1995,Chop-suey Chinese,Switzerland,Tokyo Traders,United Package,135.999999,24.000000,20
3,10371,January 1995,La maison d'Asie,France,Svensk Sjöföda AB,Speedy Express,72.960000,18.240001,6
4,10373,January 1995,Hungry Owl All-Night Grocers,Ireland,Escargots Nouveaux,Federal Shipping,678.399997,169.600000,80


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 9 columns):
order_id            247 non-null int64
long_date           247 non-null object
Customer            247 non-null object
Customer Country    247 non-null object
Supplier            247 non-null object
Shipper             247 non-null object
Amount              247 non-null float64
Discount            247 non-null float64
Quantity            247 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 17.4+ KB


In [89]:
def strptime(*args):
    return lambda x: datetime.datetime.strptime(x, *args)

def strftime(*args):
    return lambda x: datetime.datetime.strftime(x, *args)
    
    
functions = {
    'int': int,
    'str': str,
    'float': float,
    'upper': lambda x: str(x).upper,
    'lower': lambda x: str(x).lower,
    'left': lambda x, y: str(x)[:y],
    'right': lambda x, y: str(x)[-y:],
    'strptime': strptime,
    'strftime': strftime
}


date = transformations['strptime']('2016-01-01', '%Y-%m-%d')
transformations['strftime'](date, '%B %Y')

'January 2016'

In [103]:
specification = {
    "target": {
        "excel": {
            "path": r"/home/bradleyk/Downloads/Dashboard - Sales - Excel.xlsx",
            "sheet": r"Orders"
        },
        "columns": {
            "Order ID": {
                "name": "order_id",
                "transformations": ["float", "str"]
            },
            "Order Date": {
                "name": "order_date",
                "transformations": [
                    "str",
                    {"func": "strptime", "args": ['%m/%d/%Y']},
                    {"func": "strftime", "args": ['%B %Y']}
                ]
            }   
        }
    }
}

In [106]:
def get_target_type(specification):
    target = specification['target']
    
    if 'excel' in target:
        return 'excel'

def parse_excel(specification):
    path = specification['target']['excel']['path']
    sheet = specification['target']['excel']['sheet']
    columns = specification['target']['columns']
    return path, sheet, columns

target = get_target_type(specification)

if target == 'excel':
    path, sheet, columns = parse_excel(specification)
    
    with Excel(path) as excel:
        table = excel[sheet]
        
        for name, values in columns.items():
            field = table[name]
            
            if values.get('name', None) is not None:
                field.name = values.get('name', None)
                
            transformations = values.get('transformations', None)
            if transformations is not None:
                for item in transformations:
                    if isinstance(item, str):
                        func = functions[item]
                    else:
                        func = item['func']
                        args = item['args']
                        func = functions[func](*args)
                    
                    field.pipeline.append(Transformation(func))
                    
        data = table.to_frame(True)
        
data.head()

,order_id,order_date,Customer,Customer Country,Supplier,Shipper,Amount,Discount,Quantity
0,10370.0,January 1995,Chop-suey Chinese,Switzerland,Exotic Liquids,United Package,183.599999,32.400000,15
1,10370.0,January 1995,Chop-suey Chinese,Switzerland,Plutzer Lebensmittelgroßmärkte AG,United Package,798.000000,0.000000,30
2,10370.0,January 1995,Chop-suey Chinese,Switzerland,Tokyo Traders,United Package,135.999999,24.000000,20
3,10371.0,January 1995,La maison d'Asie,France,Svensk Sjöföda AB,Speedy Express,72.960000,18.240001,6
4,10373.0,January 1995,Hungry Owl All-Night Grocers,Ireland,Escargots Nouveaux,Federal Shipping,678.399997,169.600000,80
